In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

df = pd.read_csv("csv 파일") # csv 데이터셋 
df.head()

: 

In [ ]:
df.describe()

: 

In [ ]:
#dimension
df.shape

: 

In [ ]:
#결측치
df.isnull().sum()

: 

In [ ]:
# information
df.info()

: 

In [ ]:
df['text'][0]

: 

In [ ]:
import re

def apply_regular_expression(text):
     hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
     result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
     return result



: 

In [ ]:
apply_regular_expression(df['text'][0])

: 

In [ ]:
from konlpy.tag import Okt
from collections import Counter

okt = Okt()


: 

In [ ]:
apply_regular_expression(df['text'][0])

: 

In [ ]:
nouns = okt.nouns(apply_regular_expression(df['text'][0]))
nouns


: 

In [ ]:
# 말뭉치 생성
corpus = "".join(df['text'].tolist())
corpus


: 

In [ ]:
# 정규 표현식 적용
apply_regular_expression(corpus)

: 

In [ ]:
# 전체 말뭉치(corpus)에서 명사 형태소 추출
nouns = okt.nouns(apply_regular_expression(corpus))
print(nouns)

: 

In [ ]:
# 빈도 탐색
counter = Counter(nouns)

: 

In [ ]:
counter.most_common(10)

: 

In [ ]:
available_counter = Counter({x: counter[x] for x in counter if len(x) > 1})
available_counter.most_common(10)


: 

In [ ]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]


: 

In [ ]:
jeju_hotel_stopwords = ['제주', '제주도', '호텔', '리뷰', '숙소', '여행', '트립']
for word in jeju_hotel_stopwords:
    stopwords.append(word)

: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df['text'].tolist())
word_list = vect.get_feature_names_out()
count_list = bow_vect.toarray().sum(axis=0)


: 

In [ ]:
# 단어 리스트
word_list

: 

In [ ]:
# 각 단어가 전체 리뷰중에 등장한 총 횟수
count_list

: 

In [ ]:
# 각 단어의 리뷰별 등장 횟수
bow_vect.toarray()

: 

In [ ]:
bow_vect.shape

: 

In [ ]:
# "단어" - "총 등장 횟수" Matching

word_count_dict = dict(zip(word_list, count_list))
word_count_dict


: 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

: 

In [ ]:
print(tf_idf_vect.shape)

: 

In [ ]:
print(tf_idf_vect[0])

: 

In [ ]:
# 첫 번째 리뷰에서 모든 단어의 중요도 -- 0인 값까지 포함
print(tf_idf_vect[0].toarray().shape)
print(tf_idf_vect[0].toarray())

: 

In [ ]:
vect.vocabulary_

: 

In [ ]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

: 

In [ ]:
df.sample(10)

: 

In [ ]:
plt.figure(figsize=(5,5))
sns.distplot(df['rating'])

: 

In [ ]:
plt.figure(figsize=(5,5))
sns.boxplot(df['rating'])

: 

In [ ]:
df['rating'].hist()

: 

In [ ]:
def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0
    
df['y'] = df['rating'].apply(lambda x: rating_to_label(x))



: 

In [ ]:
df.head()

: 

In [ ]:
df["y"].value_counts()

: 

In [ ]:
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)

: 

In [ ]:
x_train.shape, y_train.shape

: 

In [ ]:
x_test.shape, y_test.shape

: 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression(random_state = 0)
lr.fit(x_train, y_train)

# predict in test set
y_pred = lr.predict(x_test)

: 

In [ ]:
# classification result for test set

print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))

: 

In [ ]:
# confusion matrix

from sklearn.metrics import confusion_matrix

confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()

: 

In [ ]:
df['y'].value_counts()

: 

In [ ]:
positive_random_idx = df[df['y']==1].sample(275, random_state=12).index.tolist()
negative_random_idx = df[df['y']==0].sample(275, random_state=12).index.tolist()


: 

In [ ]:
random_idx = positive_random_idx + negative_random_idx
x = tf_idf_vect[random_idx]
y = df['y'][random_idx]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)



: 

In [ ]:
x_train.shape, y_train.shape

: 

In [ ]:
x_test.shape, y_test.shape

: 

In [ ]:
lr2 = LogisticRegression(random_state = 0)
lr2.fit(x_train, y_train)
y_pred = lr2.predict(x_test)


: 

In [ ]:
# classification result for test set

print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))


: 

In [ ]:
# confusion matrix

from sklearn.metrics import confusion_matrix

confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()


: 

In [ ]:
lr2.coef_

: 

In [ ]:
# print logistic regression's coef

plt.figure(figsize=(10, 8))
plt.bar(range(len(lr2.coef_[0])), lr2.coef_[0])


: 

In [ ]:
print(sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)[:5])
print(sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)[-5:])
# enumerate: 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환함


: 

In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = False)
coef_pos_index

: 

In [ ]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer


: 

In [ ]:
from wordcloud import WordCloud

: 

In [ ]:
fontpath = 'C:/Windows/Fonts/NanumGothic.ttf'

positive_word = []
for coef in coef_pos_index: # 긍정 키워드 출력
    positive_word.append(invert_index_vectorizer[coef[1]])
wc = WordCloud(
    width = 600,
    height = 600,
    max_words=20000,
    font_path = fontpath,
    background_color='white'
).generate(str(positive_word))
plt.figure(figsize=(20, 10))
plt.imshow(wc)
plt.axis('off')

: 

In [ ]:
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])


: 

In [ ]:
negative_word = []
for coef in coef_neg_index:
    negative_word.append(invert_index_vectorizer[coef[1]])

wc = WordCloud(
width = 600,
height = 600,
max_words=20000,
font_path = fontpath,
background_color='white'
).generate(str(negative_word))
plt.figure(figsize=(20, 10))
plt.imshow(wc)
plt.axis('off')

: 

In [ ]:
for coef in coef_neg_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

: 

: 